In [ ]:
import pandas as pd

# load the dataset
file_path = '../data_gpt_labeler/final_data_sampled (1).csv'
df_sample = df.sample(n=5, random_state=42).copy()

# display the first few rows of the dataframe
df_sample.head()

,Unnamed: 0,rating,text,business_name,business_category,business_description,_id
0,848694,5,Excellent beach for family activities great su...,'Ehukai Beach Park,"['Park', 'Public beach', 'Tourist attraction']",Popular surfing beach offering massive wintert...,1.1730942640485394e+20_1605375558437
1,848706,5,My favorite Beach for surfing on Oahu North Sh...,'Ehukai Beach Park,"['Park', 'Public beach', 'Tourist attraction']",Popular surfing beach offering massive wintert...,1.1249899958787118e+20_1570685676722
2,848685,5,Usually a parking spot available and a nice sp...,'Ehukai Beach Park,"['Park', 'Public beach', 'Tourist attraction']",Popular surfing beach offering massive wintert...,1.1677373083828122e+20_1618554513347
3,848711,5,Nice small beach. Great place to watch surfers,'Ehukai Beach Park,"['Park', 'Public beach', 'Tourist attraction']",Popular surfing beach offering massive wintert...,1.0664503467931671e+20_1541146996259
4,848700,5,Awesome spot for surfing!,'Ehukai Beach Park,"['Park', 'Public beach', 'Tourist attraction']",Popular surfing beach offering massive wintert...,1.1425088661032362e+20_1612418675718


In [ ]:
!pip install langdetect

In [2]:
from textblob import TextBlob
from langdetect import detect, LangDetectException
from sentence_transformers import SentenceTransformer, util
import numpy as np
import matplotlib.pyplot as plt

model = SentenceTransformer('all-MiniLM-L6-v2')

In [3]:
import nltk
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt_tab to /Users/evan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /Users/evan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [ ]:
def calculate_interpretability_score(text):
    if not isinstance(text, str) or not text.strip():
        return 0.0

    # language detection
    try:
        if detect(text) != 'en':
            return 0.0
    except LangDetectException:
        return 0.0

    # using TextBlob for sentence tokenization and POS tagging
    try:
        blob = TextBlob(text)
    except Exception:
        return 0.0

    # coherence (sentence similarity)
    sentences = [str(s) for s in blob.sentences]
    if len(sentences) < 2:
        if len(blob.words) < 3:
            return 0.1
        else:
            return 0.5

    embeddings = model.encode(sentences, convert_to_tensor=True)
    cosine_scores = util.pytorch_cos_sim(embeddings, embeddings)

    coherence_scores = []
    for i in range(len(sentences) - 1):
        coherence_scores.append(cosine_scores[i][i+1].item())

    if not coherence_scores:
        return 0.5

    avg_coherence = np.mean(coherence_scores)
    normalized_coherence = (avg_coherence + 1) / 2

    # POS tagging for basic grammar check (as a penalty)
    pos_tags = [tag for word, tag in blob.tags]
    has_verb = any(tag.startswith('VB') for tag in pos_tags)
    has_noun = any(tag.startswith('NN') for tag in pos_tags)
    if not (has_verb and has_noun):
        normalized_coherence *= 0.5

    return normalized_coherence

In [ ]:
df_sample['interpretability_score'] = df_sample['text'].apply(calculate_interpretability_score)
df_sample.head()

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
sns.histplot(df_sample['interpretability_score'], bins=20, kde=True)
plt.title('Distribution of Interpretability Scores')
plt.xlabel('Score')
plt.ylabel('Frequency')
plt.show()